In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import json
import re
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer, average_precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, precision_score, f1_score

# Open Dataset

Choose the feature only we need

In [2]:
df = pd.read_csv('demographic.csv')
df = df.drop('index', axis=1)

df_bs = df[[
    'flag_bad',
    'de_age',
    'de_gender',
    'de_num_friends',
    'de_monthly_salary',
    'de_employment_type',
    'de_employment_duration',
    'de_education',
    'de_marital_status',
    'de_children',
    'ph_call_log_stats',
    'ph_total_contacts',
    'de_date_joined',
    'fb_last_updated_date',
    'ph_app_list',
    'fb_dob',
    'fb_relation',
    'fb_gender'
]].copy()

/tmp/ipykernel_287374/195087651.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('demographic.csv')


Here we fill null values with unstated so the model can learn it also

In [3]:
df_bs.fb_gender = df_bs.fb_gender.fillna("unstated")
df_bs.fb_relation = df_bs.fb_relation.fillna('Unstated')

For fb relation, we also prevent high cardinality by assigning some category to other that is related

In [4]:
df_bs['pending_relation'] = df_bs.fb_relation.str.contains('Pending')
df_bs.fb_relation = df_bs.fb_relation.str.replace(' (Pending)', '')
df_bs.fb_relation = df_bs.fb_relation.str.replace('In a civil union', 'Married')
df_bs.fb_relation = df_bs.fb_relation.str.replace('In a domestic partnership', 'Married')
df_bs.fb_relation = df_bs.fb_relation.str.replace('Separated', 'Divorced')

### Feature Engineering

We also create new feature: fb_age which is the difference between the date of birth in FB and date joined. For the EDA information that stated we need to find difference first, xgboost can handle it automatically and find pattern as long as we provide the root features (fb_age and the de_age)

In [5]:
df_bs['fb_age'] = ((pd.to_datetime(df.de_date_joined, dayfirst=True) - pd.to_datetime(df.fb_dob))/pd.Timedelta(days=1))/365

Then the interesting part. For app list, we extract several information about the user apps like gambling apps, clone, mod, finance risky, invest, adult and suspicious utility by using keywords. The idea behind finance risky and invest is my concern that they will use our bnpl to pay their margin or just fraudsters getting much money by more financial apps.

In [6]:
categories = {
    "gambling": ['casino', 'togel', 'prediksi', 'blackjack', 'jitu', 'mangosoft', 'zynga', 'gamble', 'dewa', 'jackpot', 'vegas', 'totolotto', 'slot', 'newflashbingo', 'luckywheel', 'roulette', 'poker', 'bingo', 'rummy', '777', '888'],
    "clone": ["clone", "dual", "parallel", "multi" , "whatscan"],
    "modded": ["mod", "hack", "vip", "apk", "patched", "crack", "cheat", "unlocked","premiumfree","vipmod","luckypatcher"],
    "finance_risky": ['loan', 'pinjam', 'tunai', 'uang', 'moneyapp', 'rupiah', 'cash', 'duit', 'kredit', 'modalku', 'julo', 'payday', 'cicil'],
    "invest": ['stockbit', 'forex', 'mirae', 'coinbase', 'binance', 'saham'],
    "adult": ['michat', 'tinder', 'bigolivegirlvideo', 'okcupid', "live", "cam", "dating", "xxx", "sexy", "hot", "escort", "porn"],
    "suspicious_utility": ['fake', 'anony', "booster", "cleaner", "locker", "battery", "optimizer", "antivirusfree", "speedup", "vpn"],
}
# Create binary columns for each category
for cat, keywords in categories.items():
    df_bs[cat] = df_bs["ph_app_list"].str.lower().apply(
        lambda x: int(any(kw in x for kw in keywords))
    )

Then the last update days relative to the date joined

In [7]:
joined = pd.to_datetime(df_bs.de_date_joined, dayfirst=True, errors="coerce").dt.tz_localize("Asia/Jakarta")
updated = pd.to_datetime(df_bs.fb_last_updated_date, errors="coerce")

df_bs["date_diff"] = ((joined - updated) / pd.Timedelta(days=1))
del joined, updated

In [8]:
def normalize_os(os_str):
    if os_str is np.nan:
        return None
    
    # --- Android ---
    if os_str.startswith("Android"):
        match = re.search(r"Android\s(\d+)(?:\.(\d+))?", os_str)
        if match:
            major = int(match.group(1))
            minor = match.group(2)

            if major < 4:
                return "Android Legacy (less than 4)"
            elif major == 4:
                if minor in ["0", "1", "2", "3"]:
                    return "Android 4 (Jelly Bean/ICS)"
                elif minor == "4":
                    return "Android 4.4 (KitKat)"
                else:
                    return "Android 4 (Jelly Bean/ICS)"  # fallback
            elif major == 5:
                return "Android 5 (Lollipop)"
            elif major == 6:
                return "Android 6 (Marshmallow)"
            elif major == 7:
                return "Android 7 (Nougat)"
            elif major >= 8:
                return "Android 8+ (Oreo & later)"
    
    # --- iOS ---
    if os_str.startswith("iOS"):
        match = re.search(r"iOS\s(\d+)", os_str)
        if match:
            major = int(match.group(1))
            if major in [6, 7]:
                return "iOS 6-7"
            elif major in [8, 9]:
                return "iOS 8-9"
            elif major >= 10:
                return "iOS 10+"
    
    return "Other/Unknown"

Then other device info. Here i take 15% time of the project to really self labelling the data, thinking the features i got will improve it. The long explanation is at device.ipynb where I used external dataset (gsmarena) to map to the phone feature. This also to prevent high cardinality in production if we use all of the phone category although xgboost can handle that in research. Alas, this time taking big research of FE left me with lesser time to explore this rich dataset. There are more things that can be tried to explore in the cleaned dataset, should be.  

Here we try to use phone price, whether its discontinued/cancelled, bucketed platform os (see normalize os function above), and sim number (back then dual sim was a new thing i believe).

In [9]:
df_device = df["ph_other_device_info"].apply(json.loads).apply(pd.Series)
df_bs = pd.concat([df_bs, df_device], axis=1)
del df_device

df_mkt_gsm = pd.read_csv('marketing_gsm_clean.csv')
df_mkt_gsm = df_mkt_gsm[[
    'device_codename',
    'brand',
    'misc_price',
    'platform_os',
    'launch_status',
    'body_sim',
]]
# Clean one error
df_mkt_gsm.misc_price = df_mkt_gsm.misc_price.replace({'<c2><a3><e2><80><89>141.60 / $<e2><80><89>149.00':'About 180 EUR'})
# Extract number only
df_mkt_gsm.misc_price = df_mkt_gsm.misc_price.str.extract(r"(\d+)")
# Get os
df_mkt_gsm.platform_os = df_mkt_gsm.platform_os.map(normalize_os)
# Get continuation status
df_mkt_gsm["discontinued"] = df_mkt_gsm.launch_status.str.contains("Discontinued|Cancelled", case=False)
# Get sim number
df_mkt_gsm.body_sim = df_mkt_gsm.body_sim.str.contains('Dual|Triple', case=False)
df_bs = df_bs.merge(df_mkt_gsm, how='left', on=['device_codename', 'brand'])

Call log stats feature engineering

In [10]:
df_call_log_stats = df["ph_call_log_stats"].apply(lambda x: json.loads(x) if isinstance(x, str) else {}).apply(pd.Series)
df_call_log_stats = df_call_log_stats.fillna(0)
df_bs = pd.concat([df_bs, df_call_log_stats], axis=1)
del df_call_log_stats

Drop the unused column and assign several things as str so it can be dummied as binary feature

In [11]:
df_bs = df_bs.drop(['brand', 'device_codename', 'ph_call_log_stats', 'de_date_joined', 'fb_last_updated_date', 'ph_app_list', 'fb_dob', 'launch_status'], axis=1)

df_bs.de_employment_type = df_bs.de_employment_type.replace(4,3)

df_bs.de_gender = df_bs.de_gender.astype(str)
df_bs.de_employment_type = df_bs.de_employment_type.astype(str)
df_bs.de_education = df_bs.de_education.astype(str)
df_bs.de_marital_status = df_bs.de_marital_status.astype(str)
df_bs.de_children = df_bs.de_children.astype(str)

Prepare before modelling by splitting data into train and test, and use stratify y so the composition by flag bad is more or less the same

In [12]:
df_bs = pd.get_dummies(df_bs)

y = df_bs.flag_bad
X = df_bs.drop('flag_bad', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
del X, y

### Modelling

In [13]:
model = XGBClassifier(
    # tree_method="hist",  # use histogram, works for both CPU and GPU
    # device="cuda:0"      # instead of gpu_id
)
model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

average_precision_score(y_test, y_proba)

0.2975730847523364

In [14]:
model = LGBMClassifier()
model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

average_precision_score(y_test, y_proba)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4256, number of negative: 28561
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001429 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3895
[LightGBM] [Info] Number of data points in the train set: 32817, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.129689 -> initscore=-1.903712
[LightGBM] [Info] Start training from score -1.903712


0.3119062971766088

As its faster and higher in baseline result, i will promote this model to parameter tuning.

In [15]:
# # Handle imbalance
# scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# # Base model
# model = LGBMClassifier(
#     random_state=42,
#     scale_pos_weight=scale_pos_weight
# )

# # Minimal parameter grid (tiny search space)
# param_grid = {
#     "num_leaves": [31, 63],         # small vs medium tree
#     "max_depth": [-1, 10],          # unlimited vs moderate
#     "learning_rate": [0.05, 0.1],   # slower vs faster
#     "n_estimators": [100, 300]      # small vs medium rounds
# }

# # Grid search with 2-fold CV for speed
# grid_search = GridSearchCV(
#     estimator=model,
#     param_grid=param_grid,
#     scoring='average_precision',
#     cv=2,          # fast
#     verbose=1,
#     n_jobs=-1
# )

# # Fit
# grid_search.fit(X_train, y_train)

# # Best model
# best_model = grid_search.best_estimator_
# print("Best parameters:", grid_search.best_params_)

# # Evaluate
# y_proba = best_model.predict_proba(X_test)[:, 1]
# ap = average_precision_score(y_test, y_proba)
# print("Average Precision Score on test:", ap)

The choosen model

In [16]:
# Handle class imbalance
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# Quick test model
model = LGBMClassifier(
    num_leaves=31,
    max_depth=10,
    learning_rate=0.1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    scale_pos_weight=scale_pos_weight
)

# Fit
model.fit(X_train, y_train)

# Predict probabilities
y_proba = model.predict_proba(X_test)[:, 1]

# Evaluate
ap = average_precision_score(y_test, y_proba)
print("Result:", ap)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4256, number of negative: 28561
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001001 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3895
[LightGBM] [Info] Number of data points in the train set: 32817, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.129689 -> initscore=-1.903712
[LightGBM] [Info] Start training from score -1.903712
Result: 0.31239447644517254


### Evaluation

We can set recall threshold around 80% to getting all the true positives more for a thumb rule. It can be adjusted also by discussion with management and credit risk team regarding the threshold and user flow each time, and if we have also other metric that measures directly to opportunity cost lost or potential gain.

In [27]:
target_recall = 0.8
best_thresh = 0
closest_diff = 1.0  # start with max difference

thresholds = np.arange(0.0, 1.01, 0.01)

for t in thresholds:
    y_pred = (y_proba >= t).astype(int)
    r = recall_score(y_test, y_pred)
    diff = abs(r - target_recall)
    if diff < closest_diff:
        closest_diff = diff
        best_thresh = t

# Final predictions using this threshold
y_pred_final = (y_proba >= best_thresh).astype(int)
recall = recall_score(y_test, y_pred_final)
precision = precision_score(y_test, y_pred_final)
f1 = f1_score(y_test, y_pred_final)

print(f"Threshold for recall ≈ {target_recall*100:.0f}%: {best_thresh:.2f}")
print(f"Recall: {recall:.3f}, Precision: {precision:.3f}, F1: {f1:.3f}")

Threshold for recall ≈ 80%: 0.36
Recall: 0.802, Precision: 0.206, F1: 0.328


Out of all our test dataset, we reject 4136 people to get that 80% recall threshold

In [28]:
# Count approvals and rejections
num_reject = y_pred_final.sum()          # predicted bad
num_approve = len(y_pred_final) - num_reject  # predicted good

print(f"Out of {len(y_pred_final)} samples:")
print(f"Approved (predicted good): {num_approve}")
print(f"Rejected (predicted bad): {num_reject}")

Out of 8205 samples:
Approved (predicted good): 4069
Rejected (predicted bad): 4136


Wrapping those all in a function.

In [29]:
def full_prediciton():
    df = pd.read_csv('demographic.csv')
    df = df.drop('index', axis=1)
    
    df_bs = df[[
        'flag_bad',
        'de_age',
        'de_gender',
        'de_num_friends',
        'de_monthly_salary',
        'de_employment_type',
        'de_employment_duration',
        'de_education',
        'de_marital_status',
        'de_children',
        'ph_call_log_stats',
        'ph_total_contacts',
        'de_date_joined',
        'fb_last_updated_date',
        'ph_app_list',
        'fb_dob',
        'fb_relation',
        'fb_gender'
    ]].copy()

    df_bs.fb_gender = df_bs.fb_gender.fillna("unstated")
    
    df_bs.fb_relation = df_bs.fb_relation.fillna('Unstated')
    df_bs['pending_relation'] = df_bs.fb_relation.str.contains('Pending')
    df_bs.fb_relation = df_bs.fb_relation.str.replace(' (Pending)', '')
    df_bs.fb_relation = df_bs.fb_relation.str.replace('In a civil union', 'Married')
    df_bs.fb_relation = df_bs.fb_relation.str.replace('In a domestic partnership', 'Married')
    df_bs.fb_relation = df_bs.fb_relation.str.replace('Separated', 'Divorced')

    df_bs['fb_age'] = ((pd.to_datetime(df.de_date_joined, dayfirst=True) - pd.to_datetime(df.fb_dob))/pd.Timedelta(days=1))/365
    categories = {
        "gambling": ['casino', 'togel', 'prediksi', 'blackjack', 'jitu', 'mangosoft', 'zynga', 'gamble', 'dewa', 'jackpot', 'vegas', 'totolotto', 'slot', 'newflashbingo', 'luckywheel', 'roulette', 'poker', 'bingo', 'rummy', '777', '888'],
        "clone": ["clone", "dual", "parallel", "multi" , "whatscan"],
        "modded": ["mod", "hack", "vip", "apk", "patched", "crack", "cheat", "unlocked","premiumfree","vipmod","luckypatcher"],
        "finance_risky": ['loan', 'pinjam', 'tunai', 'uang', 'moneyapp', 'rupiah', 'cash', 'duit', 'kredit', 'modalku', 'julo', 'payday', 'cicil'],
        "invest": ['stockbit', 'forex', 'mirae', 'coinbase', 'binance', 'saham'],
        "adult": ['michat', 'tinder', 'bigolivegirlvideo', 'okcupid', "live", "cam", "dating", "xxx", "sexy", "hot", "escort", "porn"],
        "suspicious_utility": ['fake', 'anony', "booster", "cleaner", "locker", "battery", "optimizer", "antivirusfree", "speedup", "vpn"],
    }
    
    # Create binary columns for each category
    for cat, keywords in categories.items():
        df_bs[cat] = df_bs["ph_app_list"].str.lower().apply(
            lambda x: int(any(kw in x for kw in keywords))
        )
    
    joined = pd.to_datetime(df_bs.de_date_joined, dayfirst=True, errors="coerce").dt.tz_localize("Asia/Jakarta")
    updated = pd.to_datetime(df_bs.fb_last_updated_date, errors="coerce")
    
    df_bs["date_diff"] = ((joined - updated) / pd.Timedelta(days=1))
    del joined, updated
    
    df_device = df["ph_other_device_info"].apply(json.loads).apply(pd.Series)
    df_bs = pd.concat([df_bs, df_device], axis=1)
    del df_device

    df_mkt_gsm = pd.read_csv('marketing_gsm_clean.csv')
    df_mkt_gsm = df_mkt_gsm[[
        'device_codename',
        'brand',
        'misc_price',
        'platform_os',
        'launch_status',
        'body_sim',
    ]]
    df_mkt_gsm.misc_price = df_mkt_gsm.misc_price.replace({'<c2><a3><e2><80><89>141.60 / $<e2><80><89>149.00':'About 180 EUR'})
    df_mkt_gsm.misc_price = df_mkt_gsm.misc_price.str.extract(r"(\d+)")
    df_mkt_gsm.platform_os = df_mkt_gsm.platform_os.map(normalize_os)
    df_mkt_gsm["discontinued"] = df_mkt_gsm.launch_status.str.contains("Discontinued|Cancelled", case=False)
    df_mkt_gsm.body_sim = df_mkt_gsm.body_sim.str.contains('Dual|Triple', case=False)
    df_bs = df_bs.merge(df_mkt_gsm, how='left', on=['device_codename', 'brand'])

    df_call_log_stats = df["ph_call_log_stats"].apply(lambda x: json.loads(x) if isinstance(x, str) else {}).apply(pd.Series)
    df_call_log_stats = df_call_log_stats.fillna(0)
    df_bs = pd.concat([df_bs, df_call_log_stats], axis=1)
    del df_call_log_stats
    
    df_bs = df_bs.drop(['brand', 'device_codename', 'ph_call_log_stats', 'de_date_joined', 'fb_last_updated_date', 'ph_app_list', 'fb_dob', 'launch_status'], axis=1)

    df_bs.de_employment_type = df_bs.de_employment_type.replace(4,3)
    
    df_bs.de_gender = df_bs.de_gender.astype(str)
    df_bs.de_employment_type = df_bs.de_employment_type.astype(str)
    df_bs.de_education = df_bs.de_education.astype(str)
    df_bs.de_marital_status = df_bs.de_marital_status.astype(str)
    df_bs.de_children = df_bs.de_children.astype(str)
    
    df_bs = pd.get_dummies(df_bs)
    
    y = df_bs.flag_bad
    X = df_bs.drop('flag_bad', axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    del X, y

    scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)
    model = LGBMClassifier(
        num_leaves=31,
        max_depth=10,
        learning_rate=0.1,
        n_estimators=100,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight = scale_pos_weight
    )
    model.fit(X_train, y_train)
    del X_train, y_train
    
    # y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]
    del X_test

    return {
        "avg_prrc": average_precision_score(y_test, y_proba)
    }

In [30]:
full_prediciton()

/tmp/ipykernel_287374/2249824592.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('demographic.csv')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4256, number of negative: 28561
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001230 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3895
[LightGBM] [Info] Number of data points in the train set: 32817, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.129689 -> initscore=-1.903712
[LightGBM] [Info] Start training from score -1.903712


{'avg_prrc': 0.31239447644517254}